In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException

In [2]:
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", "3X2XyB8si7yYSByRQ1lj")
    req.add_header("X-Naver-Client-Secret", "IRc7OibXu_")
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [3]:
def getNaverSearchResult(sNode, search_text, page_start, display):
    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [4]:
def getPostData(post, jsonResult):
    
    title = post['title']
    description = post['description']
    link = post['link']

    jsonResult.append({'title':title, 'description': description, 'link': link})
        
    return

In [5]:
def makeJson():

    jsonResult = []

    # 'news', 'blog', 'cafearticle', 'kin'

    sNode = 'kin'
    search_text = '소화제'
    display_count = 100
    
    jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)
    
    while ((jsonSearch != None) and (jsonSearch['display'] != 0)):
        for post in jsonSearch['items']:
            getPostData(post, jsonResult)
        
        nStart = jsonSearch['start'] + jsonSearch['display']
        
        if nStart > 1000:
            break
        
        jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
    
    with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf8') as outfile:
        retJson = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(retJson)
        
    print ('%s_naver_%s.json SAVED' % (search_text, sNode))
    
    return search_text

In [6]:
def makeLinkList(keyword):
    items = json.load(open(keyword + "_naver_kin.json", "r", encoding="utf-8"))
    links = []
    
    idx = 1
    for item in items:
        print(str(idx) + " : " + item["link"])
        links.append(item["link"])
            
        if idx == 5:
            break
            
        idx += 1
    
    return links

In [10]:
def textCrawling(links, keyword):
    
    wfile = open(os.getcwd() + "/" + keyword + "_naver_kin.txt", mode='w', encoding='utf-8')
    
    idx = 1
    for link in links:
        try:
            driver = webdriver.Chrome("/Users/leesoyeon/chromedriver")
            driver.get(link)

            print("\nnaver_kin_" + str(idx) + "\n")
            
            html = driver.page_source

            bs = BeautifulSoup(html, 'html.parser')
                
            title = bs.select("#content > div.question-content > div > div.c-heading._questionContentsArea > div.c-heading__title > div > div")
            question = bs.select("#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__content")
            answer = bs.find_all('div', {'class':'c-heading-answer__content-user'})

            wfile.write("\nnaver_kin_" + str(idx))

            printAndWriteData(wfile, title)
            printAndWriteData(wfile, question)
            printAndWriteData(wfile, answer)
        
        except Exception as e:
            print(e)
        
        finally:
            idx = idx + 1
            driver.close()
    
    wfile.close()

In [11]:
def printAndWriteData(file, textList):
    for text in textList:
        content = text.get_text().strip()
        print(content)
        file.write(content)
        
    return

In [12]:
def main():
    keyword = makeJson()
    links = makeLinkList(keyword)
    result = textCrawling(links, keyword)
    
if __name__ == "__main__":
    main()

[2019-05-15 23:34:48.566886] Url Request Success
[2019-05-15 23:34:48.841723] Url Request Success
[2019-05-15 23:34:49.194445] Url Request Success
[2019-05-15 23:34:49.652161] Url Request Success
[2019-05-15 23:34:49.994539] Url Request Success
[2019-05-15 23:34:50.474784] Url Request Success
[2019-05-15 23:34:50.819937] Url Request Success
[2019-05-15 23:34:51.043549] Url Request Success
[2019-05-15 23:34:51.154962] Url Request Success
[2019-05-15 23:34:51.388990] Url Request Success
소화제_naver_kin.json SAVED
1 : https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=7010102&docId=325949769&qb=7IaM7ZmU7KCc&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0
2 : https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70404&docId=326785913&qb=7IaM7ZmU7KCc&enc=utf8&section=kin.qna&rank=2&search_sort=0&spq=0
3 : https://kin.naver.com/qna/detail.nhn?d1id=7&dirId=70401&docId=323520901&qb=7IaM7ZmU7KCc&enc=utf8&section=kin.qna&rank=3&search_sort=0&spq=0
4 : https://kin.naver.com/qna/detail.nhn?d1id=7&dir